In [28]:
import pandas as pd
import numpy as np
import sys

import tensorflow as tf

In [29]:
class TF_Agent():
    
    def __init__(self, keys):
        self._initialize(keys)
        
    def _initialize(self, keys):
        np.random.seed(0)
        self.log_df = pd.DataFrame({key: [] for key in keys})
        self.keys = self.log_df.columns
        self.interesting = ",".join(self.log_df.columns)
        self.iter_count = 0     

    def set_batch(self, n, feed_dict):
        
        keys = list(feed_dict.keys())
        
        data_n = len(feed_dict[keys[0]])
        indices = np.random.choice(range(data_n), n, replace=False)
        
        feed_dict = {key: feed_dict[key][indices] for key in keys}
        self.feed_dict = feed_dict
    
    def run_session(self, sess):
        
        values = sess.run(eval(self.interesting),
                          feed_dict=self.feed_dict)
        
        self.recent_log = pd.Series(values, index=self.keys)
        self.log_df = self.log_df.append([self.recent_log])
        
        self.iter_count += 1
        
    def trace(self, args, one_line_text=""):
        
        iter_count = self.iter_count
        deco = ["-", "\\", "|", "/"][iter_count%4]
        
        one_line_text = "Iter: % 6d"%iter_count
        for arg in args:
            one_line_text += " %s %s: %0.9f"%(
                deco, arg, self.recent_log[arg]
            )

        sys.stdout.write("\r%s"%one_line_text)

In [16]:
normal_trian_X = pd.read_pickle("./data/normal_tr_X")
underscore_trian_X = pd.read_pickle("./data/underscore_tr_X")
train_y = pd.read_pickle("./data/tr_y")

In [42]:
rnn_size = 64
iter_n = 5000
# cases = ["normal", "underscore"]
cases = ["normal"]
tfadic = {case: TF_Agent(["loss", "learn"]) for case in cases}

for case in cases:
    
    if case == "normal":
        train_X = normal_trian_X
        squence_size = 80
    elif case == "underscore":
        train_X = underscore_trian_X
        squence_size = 160
    
    tf.reset_default_graph()
    g = tf.get_default_graph()
    tf.set_random_seed(1050554145)

    X = tf.placeholder(dtype=tf.float32, shape=[None, squence_size, 128])
    y = tf.placeholder(dtype=tf.float32, shape=[None, 2])

    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=rnn_size)
    outputs, state = tf.nn.dynamic_rnn(lstm_cell, X, dtype=tf.float32)
    logits = state[1]

    weight = tf.Variable(tf.truncated_normal([rnn_size, 2]))
    bias = tf.Variable(tf.truncated_normal([2]))

    fc = tf.matmul(logits, weight) + bias
    loss = tf.losses.softmax_cross_entropy(y, fc)
    learn = tf.train.AdamOptimizer().minimize(loss)

    initializer = tf.global_variables_initializer()

    
    sess = tf.Session(graph=g)
    sess.run(initializer)

    tfa = tfadic[case]
    for _ in range(iter_n):
        tfa.set_batch(32, {X: train_X, y: train_y})
        tfa.run_session(sess)
        tfa.trace(["loss"])

    print()
#     sess.close()